In [ ]:
# Importation de la librairie warnings et désactivation des avertissements (pour rendre les sorties code plus lisible et éviter les avertissements de version de librairies)
import warnings
warnings.filterwarnings('ignore')

# Importation des librairies pandas et numpy permettant la manipulation des données et des tableaux multidimensionnels 
import pandas as pd
import numpy as np

# Librairies de visualisation permettant de tracer différents graphes.
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Paramètrage des visualisations
sns.set(style='darkgrid')
plt.rcParams["patch.force_edgecolor"] = True

In [ ]:
# Nous devons dans un premier temps charger les données depuis notre dataset. Le dataset contient 2 fichiers 'train.csv'
# et 'test'.csv' contenant respectivement plus de 550 000 entrées et plus de 234 000 entrées.

# Nous chargons donc les données dans un tableau de données.

# Train set
train = pd.read_csv('../input/train.csv')
# Test set
test = pd.read_csv('../input/test.csv')

## Partie 1: Prétraitement des données

### Visualisation préliminaire

In [ ]:
# Regardons les 5 premières entrées ainsi que les 5 dernières pour se donner une idée de la forme que prenne les entrées.

# Ensemble d'entrainement:

# 5 premières lignes pour l'ensemble d'entrainement:
train.head(5)

In [ ]:
# 5 dernières lignes pour l'ensemble d'entrainement:
train.tail(5)

In [ ]:
# Visualisons également les types des données d'entrée ainsi que la taille exacte de l'ensemble de données.
print(train.info())
print('Shape: ',train.shape)

In [ ]:
# Explication ici: tailles des ensembles de données.
# Nombre de colonnes pour l'ensemble train: 12 => target(price) est dans l'ensemble; nous nous en servirons pour entrainer l'algorithme
# A l'inverse, 11 colomnes pour l'ensemble train (pas la target). Nous nous en servirons pour tester les performances de l'algorithme sur plusieurs modèles de régression
# Parler également du type des variables et de la signification de chacune des colomnes

In [ ]:
# Ensemble de test:

# 5 premières lignes pour l'ensemble de test:
test.head(5)

In [ ]:
# 5 dernières lignes pour l'ensemble de test:
test.tail(5)

In [ ]:
# Visualisons également les types des données d'entrée ainsi que la taille exacte.
print(test.info())
print('Shape: ',test.shape)

### Traitement de valeurs manquantes

In [ ]:
# Examinons de plus près les valeurs manquantes des ensemble des données

# Ensemble d'apprentissage

total_miss = train.isnull().sum()
perc_miss = total_miss/train.isnull().count()*100

missing_data = pd.DataFrame({'Total missing':total_miss,
                            '% missing':perc_miss})

missing_data.sort_values(by='Total missing',
                         ascending=False).head(3)

In [ ]:
# Ensemble de test

total_miss = test.isnull().sum()
perc_miss = total_miss/test.isnull().count()*100

missing_data = pd.DataFrame({'Total missing':total_miss,
                            '% missing':perc_miss})

missing_data.sort_values(by='Total missing',
                         ascending=False).head(3)

Seulement 2 colomnes: 'Product_Category_3'  et 'Product_Category_2' possèdent des valeurs manquantes. La colonne 'Product_Category_3' possède près de 70% de valeurs manquantes et  la colomne 'Product_Category_2' plus de 30% de données manquantes.

In [ ]:
# Nous devons maintenant nous occuper de ces données manquantes.
# Plusieurs choix s'offrent à nous. Nous pourrions simplement supprimer les lignes possèdant une valeur manquante.
# Cependant, ce choix apparait ici très mauvais car près de 70% des donnnées sont manquantes pour la colomne 'Product_Category_3'.
# Compte tenu du fort pourcentage de données manquante pour cette colomne, nous choisissons de la supprimer de l'ensemble d'apprentissage et de l'ensemble de test.
half_count_train = len(train) / 2 # Cette ligne est un seuil
half_count_test = len(test) / 2 # Cette ligne est un seuil

train = train.dropna(thresh=half_count_train,axis=1) # Cette ligne supprime les colomne comportant plus de 50% de données manquantes
test = test.dropna(thresh=half_count_test,axis=1) # Cette ligne supprime les colomne comportant plus de 50% de données manquantes

In [ ]:
# Regardons de nouveau un échantillon de nos ensembles.
train.head()

In [ ]:
test.head()

Nous avons supprimer la colonne 'Product_Category_3' de nos 2 tableaux de données 'train' et 'test'. Cependant, nous l'avons vu, la colone 'Product_Category_2' possède plus de 30% de données manquantes; pourcentage important mais qui reste inférieur au seuil fixé de 50%.
Ainsi cette colonne demeure dans les 2 tableaux de données. Les données manquantes de cette colonne ne peuvent cependant pas rester inchangées dans les tableaux. 

Intéressons nous donc à cette colonne plus en détail pour déterminer une technique de remplacement des données manquantes.

In [ ]:
print(train.Product_Category_2)

Affichons le minimum et le maximum de cette colonne pour l'ensemble de données d'entrainement et de test.

In [ ]:
print(train.Product_Category_2.min())
print(train.Product_Category_2.max())

In [ ]:
print(test.Product_Category_2.min())
print(test.Product_Category_2.max())

Il semble que les valeurs de cette colonnes sont comprises entre 2 et 18. Nous pouvons remplacer les valeurs manquantes par la moyenne des valeurs non nulles de la colonne

In [ ]:
# Commencons par calculer cette moyenne
avg_train = train.Product_Category_2.mean()
avg_test = test.Product_Category_2.mean()

print(avg_train)
print(avg_test)

In [ ]:
# Remplacons les valeurs manquantes de l'ensemble d'apprentissage et de l'ensemble de test par les moyennes repectives 
train['Product_Category_2'].fillna(avg_train, inplace=True)
test['Product_Category_2'].fillna(avg_test, inplace=True)

Vérifions que les données manquantes ont bien été remplacées comme désiré.

In [ ]:
print(train.Product_Category_2)

In [ ]:
print(test.Product_Category_2)

Les données manquantes ont bien été remplacées. Il nous faut maintenant continuer le processus de prétraitement des données en analysant davantage les colonnes.

### Etude et représentation des colonnes

#### User_ID et Product_ID

Ces données sont des identifiants et doivent ainsi être traitées comme données catégorie.
Nous devons connaitre un peu mieux la distribution de ces données pour déterminer le nombre de catégories présentes.

In [ ]:
# Train set
unique_users = len(train.User_ID.unique())
unique_products = len(train.Product_ID.unique())
print('There are {} unique users and {} unique products in the train set'.format(unique_users, unique_products))

In [ ]:
# Test set
unique_users = len(test.User_ID.unique())
unique_products = len(test.Product_ID.unique())
print('There are {} unique users and {} unique products in the test set'.format(unique_users, unique_products))

Rappelons que notre ensemble de données contenait plus d'un demi million d'entrées . Ainsi il y'a environ 100 lignes pour chaque utilisateur et 160 lignes pour chaque produit. Ces colonnes sont donc importantes et ne doivent  pas être supprimer dans le cadre d'une analyse ciblée sur un échantillon de consommateurs.
En effet, nous avons suffisament d'informations dans notre ensemble de données pour apprendre des identifiants consommateurs et des identifiants produits.
Cependant, le nombre de catégories est très important et il serait compliqué de simplement encoder chacune des catégories  par des vecteurs type one-hot encoder. (remplacement de chaque catégorie par un vecteur de taille n où n est le nombre de catégories et où chaque entrée du vecteur est représentée par un 1 ou un 0 en fonction de la présence ou non de la catégorie considéree.)

De plus, ces colonnes ne permettent pas d'avoir une généralisation permettant de cibler globalement un large éventail de consommateurs (au contraire de l'age, du statut marital, ...) puisque chaque entrée est spécifique à un consommateur dans le cadre de l'identifiant consommateur et spécifique à un seul produit pour l'identifiant produit.

Nous décidons ainsi de supprimer ces colonnes de notre analyse.

In [ ]:
# Drop the columns Product_Id and User_Id in the train and test set
train = train.drop(columns="User_ID")
train = train.drop(columns="Product_ID")
test = test.drop(columns="User_ID")
test = test.drop(columns="Product_ID")

Image du procédé d'encodage 
![](https://cdn-images-1.medium.com/max/1368/0*T5jaa2othYfXZX9W.)


#### Gender, Age, Occupation, and City_Category

Ces 4 colonnes sont relativement simples dans une optique de prétraitement. 

Commencons par visualiser le détail des catégories pour chacune des 4 colonnes.

In [ ]:
for col_name in ['Gender', 'Age', 'Occupation', 'City_Category']:
    print(sorted(train[col_name].unique()))
    print(sorted(test[col_name].unique()))

Les catégories de chaque colonne sont identiques pour l'ensemble d'apprentissage et l'ensemble de test. Le nombre de catégories est plutot faible (maximum 21 pour la colonne 'Occupation')  
Ces colonnes sont toutes des catégories qui peuvent être encodées via des vecteurs one-hot encoded. Cette partie d'encodage sera effectuée dans la partie suivante. 

#### Marital_Status

Cette colonne est également catégorique. En effet, la valeur de chacune des entrées pour cette colonne peut être soit 0 (individu non marié) soit 1 (individu marié).  

In [ ]:
train['Marital_Status'].unique()

#### Stay_In_Current_City_Years

Cette colonne représente le nombre d'années passés par un consommateur dans la même ville. Nous pouvons imaginer une corrélation entre le nombre d'années passées dans la même vilel et le prix du panier d'un consommateur lors du black Friday. Un consommateur ayant longuement vécu dans la même ville peut posséder davantage de ressources financières qu'un nouvel habitant.

In [ ]:
train['Stay_In_Current_City_Years'].unique()

Nous pouvons observer que le type des valeurs de cette colonne sont des chaines de caractères et non des entiers comme nous pourrions le penser.
Ceci s'explique par le fait qu'une de ces valeurs représente en fait une catégorie (à savoir 4+). Pour éviter la création de nouvelles catégories, nous faisons le  choix de transformer le type de cette variable en entier.  Cette opération permettra de linéariser les données de cette colonne de façon à ce qu'il y'ait une relation d'ordre entre celles-ci.

In [ ]:
# Train set
# We first need to convert the '4+' value to simply '4'
train['Stay_In_Current_City_Years'] = [x.strip().replace('4+', '4') for x in train['Stay_In_Current_City_Years']]
# Then we need to replace the object type values by integers                                                                                       
train['Stay_In_Current_City_Years'] = train['Stay_In_Current_City_Years'].astype(str).astype(int)
print(train['Stay_In_Current_City_Years'])

# Test set
test['Stay_In_Current_City_Years'] = [x.strip().replace('4+', '4') for x in test['Stay_In_Current_City_Years']]
# Then we need to replace the object type values by integers                                                                                       
test['Stay_In_Current_City_Years'] = test['Stay_In_Current_City_Years'].astype(str).astype(int)
print(test['Stay_In_Current_City_Years'])

#### Product_Category

Nous avons préalablement supprimer la colonne Product_Category_3 car celle-ci contenait un nombre de données manquantes suppérieur au seuil que nous avons fixé à 50%. Les catégories de produit 1 et 2 quant à elles demeurent dans notre ensemble de données. Cependant au même titre que l'identifiant produit ou l'identifiant consommateur, ces informations ne permettent pas une bonne généralisation et s'inscrivent davantage dans le cadre d'une analyse ciblée où une analyse rétrospective (une fois que nous connaissons la valeur du montant du panier des consommateurs, ces données sont particulierement utiles pour une analyse de suivi client.) 
Dans le cadre de notre étude, nous cherchons à utiliser les indicateurs bas niveaux comme l'age, le sexe, le statut marital, l'occupation ou encore le nombre d'années passées dans la ville de résidence actuelle des  consommateurs  pour dégager des tendances et faire un ciblage global de types de consommateurs.

Nous décidons ainsi de supprimer également les colonnes Product_Category_1 et Product_Category_2.

In [ ]:
# Drop the columns Product_Category_1 and Product_Category_2 in the train and test set
train = train.drop(columns="Product_Category_1")
train = train.drop(columns="Product_Category_2")
test = test.drop(columns="Product_Category_1")
test = test.drop(columns="Product_Category_2")

## Partie 2: Visualisation et encodage

### Visualisation

Dans cette partie, nous allons analyser graphiquement les données pour avoir une meilleure vue d'ensemble de celles-ci et pour dégager les variables les plus importantes pour la détermination du montant du panier d'achat d'un consommateur.

#### Ensemble d'apprentissage

In [ ]:
for col_name in train.columns:
    print(col_name, len(train[col_name].unique()))

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=[15, 15])

train['Gender'].value_counts().plot(kind='barh', ax=axes[0,0], title='Gender')
train['Age'].value_counts().plot(kind='barh', ax=axes[0,1], title='Age')
train['City_Category'].value_counts().plot(kind='barh', ax=axes[1,0], title='City_Category')
train['Marital_Status'].value_counts().plot(kind='barh', ax=axes[1,1], title='Marital_Status')
train['Occupation'].value_counts().plot(kind='barh', ax=axes[2,0], title='Occupation')
train['Stay_In_Current_City_Years'].value_counts().plot(kind='barh', ax=axes[2,1], title='Stay_In_Current_City_Years')

#### Ensemble de test

In [ ]:
for col_name in test.columns:
    print(col_name, len(test[col_name].unique()))

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=[15, 15])

test['Gender'].value_counts().plot(kind='barh', ax=axes[0,0], title='Gender')
test['Age'].value_counts().plot(kind='barh', ax=axes[0,1], title='Age')
test['City_Category'].value_counts().plot(kind='barh', ax=axes[1,0], title='City_Category')
test['Marital_Status'].value_counts().plot(kind='barh', ax=axes[1,1], title='Marital_Status')
test['Occupation'].value_counts().plot(kind='barh', ax=axes[2,0], title='Occupation')
test['Stay_In_Current_City_Years'].value_counts().plot(kind='barh', ax=axes[2,1], title='Stay_In_Current_City_Years')

La distribution des données de l'ensemble de test est assez similaire à celle de l'ensemble d'apprentissage. Ceci laisse supposer de bons résultats lors de la phase d'analyse prédictive.
Sur la base de la visualisation effectuée, nous sommes en mesure de dégager les resultats suivants:

* Les Hommes consomment plus que les Femmes
* Les Célibataires consomment davantage que les personnes mariées
* Les Clients résidant dans une ville de niveau social intermédiaire dépensent plus
* Les Clients ayant vécu 1 an dans leur ville de résidence actuelle dépensent plus
* Les Clients de 29 à 36 ans sont les plus dépensiers


Ces simples relations permettent de dégager des tendances et d'établie un profil consommateur à cibler en priorité lors de campagnes marketing.
Ici, compte tenu des résultats établis, le consommateur parfait serait un homme célibataire d'une trentaine d'années vivant dans une ville de niveau social intermédiaire depuis 1 an.

**Le consommateur parfait**
![](http://1.bp.blogspot.com/-a1RLc8TDlYc/UcrD6y7jD2I/AAAAAAAABXc/tmtZDOwyckg/s1600/blond+barbe.jpg)

### Encodage

Pour que l'algorithme que nous réaliserons dans la section suivante n'apprenne pas des relations d'ordre inexisante entre les variables catégoriques (genre, statut marital, tranche d'age...), il nous faut encoder ces variables d'après la méthode de one hot encoding explicitée précedemment.

In [ ]:
# Rappelons que les variables catégoriques sont les suivantes: Age, Gender, Occupation, Ciy_Category, Marital_Status, Product Category 1, ¨Product Category 2
train.dtypes

Les variables 'Marital_Status' et 'Gender' peuvent être représentées simplement par des données binaires (0 ou 1) et ne nécessitent donc pas d'être one-hot-encoded. Cependant, les autres variables doivent subir ce traitement pour ne pas biaiser l'apprentissage de l'algorithme.

In [ ]:
# Train set
# Nous remplacons tout d'abord les valeurs de la colonne 'Gender' initialement égales à 'F' par 0 et 'M' par 1
# Cette opération n'est pas nécessaire pour la colonne 'Marital_Status' puisque ces valeurs sont déjà sous cette forme
train.Gender = np.where(str(train.Gender)=='M',1,0) # Femelle: 0, Male: 1
test.Gender = np.where(str(test.Gender)=='M',1,0) # Femelle: 0, Male: 1

# Nous encodons ensuites les variables catégoriques (Age, City_Category, Occupation)
train_Age = pd.get_dummies(train.Age)
train_CC = pd.get_dummies(train.City_Category)
train_Occup = pd.get_dummies(train.Occupation)
test_Age = pd.get_dummies(test.Age)
test_CC = pd.get_dummies(test.City_Category)
test_Occup = pd.get_dummies(test.Occupation)

# Et nous remplacons les variables catégoriques initiales par les variables encodées
train_encoded = pd.concat([train_Age,train_CC,train_Occup,train],axis=1)
train_encoded.drop(['Age','City_Category','Occupation'],axis=1,inplace=True)
test_encoded = pd.concat([test_Age,test_CC,test_Occup,test],axis=1)
test_encoded.drop(['Age','City_Category','Occupation'],axis=1,inplace=True)
print(train_encoded)
print(test_encoded)

Nos colonnes sont maintenant bien encodées. Les données sont ainsi prêtes pour être analysées et utilisées pour l'apprentissage de notre algorithme.

## Partie 3: Algorithme

Différents algorithmes de régression permettent de donner de bons résultats (Seulement si la phase de prétraitement a bien été opéree).
Nous implémenterons l'algorithme 'forêt d'arbres décisionnels' plus connu en anglais sous le nom de 'Random Forest'

Pour commencer, nous pouvons separer les variables indépendantes de la variable dépendante (variable cible).

In [ ]:
X_train = train_encoded.iloc[:, :-1].values # Prend toutes les colonnes sauf la dernière (variable cible)
y_train = train_encoded.iloc[:,-1].values # Prend la dernière colonne (variable cible)

# Pour présire les nouveaux résultats
X_test = test_encoded # Prend toutes les colonnes sauf la dernière (variable cible)

In [ ]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0) # nous utilisons une foret de 10 arbres décisionnels, random_state supprime la variabilité des résultats
regressor.fit(X_train, y_train)

In [ ]:
# Predict new results on the test set
prediction = np.round(regressor.predict(X_test))
print(prediction)
# Reshape prediction vector into a 2D matrix
prediction = prediction.reshape(len(prediction), 1)

# concatenate X_test with prediction 
dataTest = np.concatenate((X_test, prediction), axis = 1)
print(dataTest)

Nous avons implémémenté un algorithme nous permettant de prévoir le montant du panier d'un consommateurs connaissant son age, son sexe, son statut marital et autres indicateurs utilisés.
Nous pouvons maintenant nous pencher sur ces résultats et déterminer notamment les critères qui influencent le plus le montant d'un panier consommateur lors du Black Friday.

In [ ]:
# Feature importance 
f_im = regressor.feature_importances_.round(3)
ser_rank = pd.Series(f_im,index=test_encoded.columns).sort_values(ascending=False)

plt.figure()
sns.barplot(y=ser_rank.index,x=ser_rank.values,palette='deep')
plt.xlabel('relative importance')

Nous pouvons clairement voir que le temps passé dans la ville de résidence actuelle est le critère majeur dans l'apprentissage de l'algorithme (près de 25% de l'apprentissage est du à ce critère) suivi par le statut marital ( ~11 % de l'apprentissage total) puis l'age du consommateur ainsi que la catégorie social de sa ville de résidence.

## New part

Testons maintenant la performance de notre algorithme en comparant les résultats obtenus à la visualisation effectuée précédemment. 
Pour cela, nous pouvons afficher les données d'entrées pour différentes valeurs de sortie de l'algorithme (prix du panier consommateur) et les comparer  aux conclusions faites post visualisation.



In [ ]:
# Options pour afficher davantage de colonnes en sortie
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Option pour avoir des résultats consistants
np.random.seed(0)

# Pour faciliter la manipulation des données, repartons de l'ensemble de test duquel nous avons supprimé les colonnes
# inutiles à notre analyse. Ajoutons simplement une colonne prediction à cet ensemble.
test['prediction'] = prediction
# Nous prenons 10 lignes de l'ensemble de test et les ordonnons selon la valeur de la colonne prédiction 
# (par ordre décroissant)
print(test.sample(10).sort_values(by='prediction',ascending=False))


> ##  Conclusion

Nous avons implémenter un algorithme nous permettant de prédire le montant du panier d'un consommateur connaissant son sexe, statut marital, age, occupation et nombre d'années passées dans la ville de résidence actuelle.
Cet algorithme se base sur des indicateurs bas niveaux facilement identifiables et pourra permettre d'estimer la potentialité à l'achat d'un consommateur. 
Dans la partie visualisation, nous avons également pu dresser le profil du consommateur parfait à cibler en priorité lors de campagnes marketing notamment (Black Friday ou autres).
Une analyse ciblée ou rétrospective qui prendrait en compte l'ensemble des indicateurs de l'ensemble de données utilisé pourrait venir compléter cette étude et déterminer notamment les produits générant le plus de ventes.